# Calibrating bias images

The purpose of calibrating bias images is three-fold:

+ Subtract overscan if you have decided your science will be better if you subtract overscan. See [this discussion of overscan]() for some guidance. 
+ Trim the overscan region off of the image if it is present, regardless of whether you have chosen to subtract the overscan.
+ Combine the bias images into a "combined" bias to be used in calibrating the rest of the images. The purpose of combining several images is to reduce as much as possible the read noise in the combined bias.

The approach in this notebook will be to reduce a single image, look at the effects the reduction step had on that image and then demonstrate how to calibrate a folder containing several images of that type.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import os

from astropy.nddata import CCDData
from astropy.visualization import hist
import ccdproc as ccdp
import matplotlib.pyplot as plt
import numpy as np

from convenience_functions import show_image

download_base_url = 'http://physics.mnstate.edu/craig/ccd-guide/'

 ## Decide where to put your calibrated images
 
 Though it is possible to overwrite your raw data with calibrated images that is a bad idea. Here we define a folder called `reduced` that will contain the calibrated data and create it if it doesn't exist.

In [ ]:
calibrated_data = Path('.', 'reduced')
calibrated_data.mkdir(exists_ok=True)

## Example with overscan subtraction

### Data for this example

The data for this example can be downloaded from http://www.stsci.edu/~etollerud/python_imred_data.tar

In [ ]:
files = ccdp.ImageFileCollection('python_imred_data')

In [ ]:
files.summary['file', 'imagetyp', 'filter', 'exptime']

In [ ]:
p = Path('.')
p

In [ ]:
os.listdir(files.location)

In [ ]:
darks_only = ccdp.ImageFileCollection('python_imred_data/darks/')
darks_only.summary['file', 'imagetyp', 'exptime']

### Subtract and then trim the overscan (one sample image)

Please see the discussion of this camera in [the overscan notebook](01.08-Overscan.ipynb#Case-1:-Cryogenically-cooled-Large-Format-Camera-(LFC)-at-Palomar) for the appropriate overscan regioin to use for this camera. Note, in particular, that it differs from the the value given in the `BIASSEC` keyword in the header of the images.

The astropy affiliated package [ccdproc](https://ccdproc.readthedocs.io) provides two useful functions here:

+ `subtract_overscan` for subtracting the overscan from the image, and 
+ `trim_image` for trimming off the overscan.

First, let's see what the values of `BIASSEC` which sometimes (but not always) indicates that there is is overscan and which part of the chip is the overscan, and `CCDSEC`, which is sometimes, but not always present, and indicates which part of the chip light hit.

Note that neither of these are standard; sometimes, for example, `trimsec` is used instead of `ccdsec`, and there are likely other variants. Some images may have neither keyword in the header. That does not necessary indicate that ovserscan isn't present. The best advice is to carefully check the documentation for the camer you are using. 

## NEED TO FIX BECAUSE APPARENTLY THE DATASEC IS MEANINGLESS

In [ ]:
files.summary['file', 'imagetyp', 'biassec', 'ccdsec', 'datasec'][0]

The fits header claims the overscan extends from the 2049th column to the end of the image (this is one-based indexing) and that the part of the image exposed to light extends over all rows and from the first column to the 2048$^{th}$ column (again, this is one-indexed).

Unfortunately, there are two differences between FITS and Python in terms of indexing:

+ Python indexes are zero-based (i.e. numbering starts at zero), FITS indexes are one-based (i.e. numbering starts at 1).
+ The *order* of the indexes is swapped.

For example, the **FITS** representation of the part of the chip exposed to light is `[1:2048,1:4128]`. To access that part of the data from a numpy array in **Python**, switch the order so that the indexing looks like this: `[0:4128, 0:2048]` (or, more compactly `[:, :2048]`). Note that the *ending* indexes given here for python are correct because the second part of a range (after the colon) is *not included* in the array slice. For example, `0:2048` starts at 0 (the first pixel) and goes up to but not including 2048, so the last pixel included is `2047` (the 2048$^{th}$ pixel).

Using `subtract_overscan` is reasonably straightforward, as shown in the cell below.

In [ ]:
raw_biases = files.files_filtered(include_path=True, imagetyp='BIAS')

In [ ]:
first_bias = CCDData.read(raw_biases[0], unit='adu')

In [ ]:
bias_overscan_subtracted = ccdp.subtract_overscan(first_bias, overscan=first_bias[:, 2055:], median=True)

Next, we trim off the full overscan region (not just the part we used for subtracting overscan)

In [ ]:
trimmed_bias = ccdp.trim_image(bias_overscan_subtracted[:, :2048])

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

show_image(first_bias.data, cmap='gray', ax=ax1, fig=fig)
ax1.set_title('Raw bias')
show_image(trimmed_bias.data, cmap='gray', ax=ax2, fig=fig)
ax2.set_title('Bias, overscan subtracted and trimmed')

### Discussion

Visually, the images look nearly identical before and after calibration. The only prominent difference is a shift in the pixel values, as one would expect from subtracting the same value from each pixel in an image. It simply shifts the zero point.

### Processing a folder of bias images

Processing each of the bias images individually would be tedious, at best. Instead, we can use the `ImageFileCollection` we created above to loop over only the bias images, saving each in the folder `calibrated_data`.

In [ ]:
for ccd, file_name in files.ccds(imagetyp='BIAS', 
                                 ccd_kwargs={'unit': 'adu'},
                                 return_fname=True
                                ):
    print('foo', file_name)
    ccd = ccdp.subtract_overscan(ccd, overscan=first_bias[:, 2055:], median=True)
    ccd = ccdp.trim_image(ccd[:, :2048])
    ccd.write(calibrated_data / file_name)

Let's check that we really did get the images we expect by creating an `ImageFileCollection` for the reduced folder and displaying the size of each image. We are expecting the images to be 2048 × 4148, and that there will be the same number of reduced bias images as input bias images (six).

In [ ]:
reduced_images = ccdp.ImageFileCollection('reduced')
reduced_images.summary['file', 'imagetyp', 'naxis1', 'naxis2']

## Example without overscan subtraction